# 02 — Query Demo (Answer from Context + Citations)
Ask a question like 'Where did we define precision vs. recall?' and get (deck, page) cites.

## Load store & model (auto-CUDA on Deck)

In [29]:
import os

# Tell Hugging Face to skip TensorFlow/Flax so they never import TensorFlow (TF).
os.environ["TRANSFORMERS_NO_TF"] = "1"
os.environ["TRANSFORMERS_NO_FLAX"] = "1"

# Quiet TF logs if something still pulls it in.
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"  # 1=INFO, 2=WARNING, 3=ERROR


# Ipmorts

In [30]:
# imports
import os, sys, json
from pathlib import Path

# If the notebook is inside .../SlideHunt/notebooks, step up to the repo root
repo_root = Path.cwd().parent if Path.cwd().name.lower() == "notebooks" else Path.cwd()
sys.path.insert(0, str(repo_root))  # make repo root's importable

from nb01_setup_and_ingest import search  

In [ ]:
# Point this ONCE to the repo root (hardcode or env var)
BASE = Path(os.getenv("SLIDEHUNT", r"C:\place path\to the repo here")).resolve()

INDEX_PATH = BASE / "data" / "faiss" / "canvas.index"
FACTS_PATH = BASE / "data" / "faiss" / "facts.json"

print("CWD:", Path.cwd())
print("Reading:", INDEX_PATH)
print("Reading:", FACTS_PATH)

# Load FAISS index (can't use open(); use faiss helper)
#index = faiss.read_index(str(INDEX_PATH))

# Load facts/metas with plain open()
with open(FACTS_PATH, "r", encoding="utf-8") as file:
    data = json.load(file)
facts, metas = data["facts"], data["metas"]

print("Loaded:", len(facts), "facts")


CWD: c:\Users\MINA-G\SlideHunter\notebooks
Reading: C:\Users\MINA-G\SlideHunter\data\faiss\canvas.index
Reading: C:\Users\MINA-G\SlideHunter\data\faiss\facts.json
Loaded: 324 facts


In [33]:
# 7) Try it out with some pre-test test-prompts
tests = [
    "Where did we define precision vs. recall?",
    "tips for a resume and cover letter?",
    "What lecture slides did we learn about control flow?",
  ]
for q in tests:
    scope, hits = search(q, k=4, scope="auto")
    print(f"\nQ: {q}   [scope={scope}]")
    if not hits: print("  (no hits)"); continue
    for h in hits:
        m = h["meta"]
        cite = f"{m['course_name']} > {m['module_name']} > {m['item_title']} ({m['type']})"
        if m.get("url"): cite += f"  [{m['url']}]"
        print(f"  {h['score']:.3f} :: {cite}")



Q: Where did we define precision vs. recall?   [scope=technical]
  0.382 :: IF '25 Data Science Cohort A > P2W3 (6/23-6/27) Classification Algorithms > 💻 W3D2 (6/24) Logistic Regression Accuracy Metrics (Page)  [https://tkh.instructure.com/courses/172/pages/w3d2-6-slash-24-logistic-regression-accuracy-metrics]
  0.306 :: Foundations '25 Data Science > Week 5:  Statistics(Feb. 24th- Feb. 27th) > What is Data Science? (Page)  [https://tkh.instructure.com/courses/165/pages/what-is-data-science]
  0.276 :: IF '25 Data Science Cohort A > P2W11 (8/18-8/22) Agents & End of Phase Project > 💻 W11D1 (8/18) Applied LLM Review & AI Agents (Page)  [https://tkh.instructure.com/courses/172/pages/w11d1-8-slash-18-applied-llm-review-and-ai-agents]
  0.263 :: IF '25 Data Science Cohort A > P2W9 (8/4-8/8) NLP Foundations & Transformers > 📚 P2W9 Overview & Lesson Plan (Page)  [https://tkh.instructure.com/courses/172/pages/p2w9-overview-and-lesson-plan]

Q: tips for a resume and cover letter?   [scope=car